## Debate Dojo

You are debating an AI Opponent. However, there is a second AI (The Referee) watching.

The Twist: If you (the Human) make a weak argument, use a logical fallacy, or get too emotional, the Referee blows the whistle (pauses the graph).

The Human-In-the-Loop Element: The graph stops. The Referee tells you why your argument was weak. You—the human—must then rewrite your argument to meet the quality bar. Only then does the graph resume and let the Opponent reply.

The Value: You are forced to improve your communication skills in real-time.

#### The Blueprint

1. The State (The Shared Memory)

- topic: The subject (e.g., "Universal Basic Income").

- history: The conversation transcript.

- last_human_argument: What you just said.

- argument_score: 1-10 rating of your logic.

- critique: The Referee's feedback.



2. The Nodes (The Workers)

- Node A: The Referee (Evaluator): Reads your input. Scores it. Checks for fallacies (Ad Hominem, Straw Man).

- Node B: The Coach (Feedback): If score < 7, generates a helpful tip. "You attacked the person, not the idea. Try focusing on the economic impact."

- Node C: The Opponent (Debater): If score >= 7, generates a counter-argument to keep the debate going.



3. The Flow (The Logic)

- Start: User submits an argument.

- Referee: Grades it.

- Router (Conditional Edge):

    - If Weak (Score < 7): Route to Coach. Then PAUSE (Interrupt).

        - HITL Action: Human reads feedback, edits their argument, and updates the state.

        - Resume: Loop back to Referee to grade the new argument.

    - If Strong (Score >= 7): Route to Opponent.



4. Opponent: Replies.

In [ ]:
import os
from typing import TypedDict, List, Annotated
import operator
from dotenv import load_dotenv

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field


In [ ]:
# Load Env file
load_dotenv()

# Pass API Key to the model

# Detbater (Fast, Conversational)
debater_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY")
    )


# Referee (Strict, Analytical)
referee_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0
    )

In [ ]:
# The State

class DojoState(TypedDict):
    topic: str
    messages: Annotated[List[BaseMessage], operator.add] #Appends new msgs to history
    latest_human_input: str
    score: int
    feedback: str

In [ ]:
# Define the Schema 

class RefereeEvaluation(BaseModel):
    """Structured output for the debate referee."""
    score: int = Field(description="A score from 1-10 rating the strength and logic of the argument.")
    feedback: str = Field(description="Concise, constructive feedback explaining the score and noting any fallacies.")
    fallacy_detected: bool = Field(description="True if a fallacy (Ad Hominem, Straw Man, etc.) was detected, False otherwise.")

#### The Nodes (Workers)

In [ ]:
# The Nodes (Workers)

# Referee Node

def referee_node(state: DojoState):
    """The Judge: Grades the human's latest argument using structured output."""
    human_arg = state["latest_human_input"]

    print(f"\n---REFEREE ANALYZING: '{human_arg}' ---")

    # Create a specialized LLM wrapper that ONLY returns our Pydantic object
    structured_llm = referee_llm.with_structured_output(RefereeEvaluation)

    prompt = f"""
    You are a debate referee. 
    Analyze the user's argument "{human_arg}" for the topic: {state["topic"]}.

    Evaluate strictly on the following criteria:
    1. Logical fallacies (Ad Hominem, Straw Man, Slippery Slope, etc.).
    2. Clarity and relevance.

    Be struct. High scores (8+) require perfect logic. 
    """

    # Invoke. The result is NOT a string message; it is a RefereeEvaluation object
    result: RefereeEvaluation = structured_llm.invoke([
        SystemMessage(content=prompt),
        HumanMessage(content=human_arg)
        ])
   
    return {
        "score": result.score,
        "feedback": result.feedback,
        "fallacy_detected": result.fallacy_detected
    }

In [ ]:
# Coach Node

def coach_node(state: DojoState):
    """The Coach: Tells the user why they paused.."""
    # We don't need an LLM here, just logic to display the feedback.
    # In an UI, this would send message to the frontend. 

    print(f"\n✋ PAUSE! WEAK ARGUMENT (Score: {state['score']}/10)")
    print(f"💡 COACH SAYS: {state['feedback']}")
    print("👉 Please refine your argument and try again.")
    
    # we return nothing because we are just preparing for interruption
    return {}

In [ ]:
# Opponent Node

def opponent_node(state: DojoState):
    """The Opponent: Generates a counter-argument if the user argument logic was sound."""
    print(f"\n Opponent Responds...")

    # Context: the History + the approved new argument
    msgs = state["messages"] + [HumanMessage(content=state["latest_human_input"])]

    system_prompt = f"You are debating the topic: {state['topic']}. Be short, witty, and logical. Disagree with the user."

    response = debater_llm.invoke([
        SystemMessage(content=system_prompt)] + msgs)

    return {"messages": [response]}

#### The Graph and the Loop

We wire the nodes together and define the interruption.
We want to route tothe coach, and then stop after the coach delivers the bad news, so user can fix it.
Actually, the cleanest pattern is to interrupt before we loop back to the referee.

In [ ]:
# The Graph & the Loop
workflow = StateGraph(DojoState)

# Add nodes
workflow.add_node("referee", referee_node)
workflow.add_node("coach", coach_node)
workflow.add_node("opponent", opponent_node)

# Entry Point
workflow.set_entry_point("referee")

# Router Logic: Where do we do after the Referee grades?
def router(state: DojoState):
    if state["score"] >= 7:
        return "approved"
    else:
        return "rejected"

workflow.add_conditional_edges(
    "referee", 
    router,
    {
        "approved": "opponent",
        "rejected": "coach"
    }
)

# If rejected, we go to Coach
# After Coach, we want to STOP. The user must input new text.
# Then we want to go back to the Referee to grade new text
workflow.add_edge("coach", "referee")

# If approved, we go to Opponent, then we end. (waiting for next turn)
workflow.add_edge("opponent", END)

# Memory is critical for Human-in-the-Loop
checkpointer = MemorySaver()

# We interrupt before the "referree" runs, if we are coming from the "coach"
# However a simpler way in LangGraph is to use the "interrupt" after the coaach node.
app = workflow.compile(
    checkpointer=checkpointer,
    interrupt_after=["coach"]
)

In [ ]:
def chat_with_dojo():
    # 1. Configuration
    # We use a static thread_id so the bot remembers the context of the debate
    config = {"configurable": {"thread_id": "live_user_1"}}
    
    print("\n--- 🥋 WELCOME TO THE SOCRATES DOJO 🥋 ---")
    topic = input("Enter a topic to debate (e.g., 'AI is dangerous'): ")
    print(f"📝 TOPIC SET: {topic}")
    print("Go ahead! Make your opening argument.\n")

    # 2. The Interactive Loop
    while True:
        # Get User Input
        user_input = input("👤 YOU: ")
        if user_input.lower() in ["quit", "exit"]:
            print("Exiting Dojo...")
            break

        # 3. Check Graph Status: Are we fixing a mistake or starting a new turn?
        snapshot = app.get_state(config)
        
        if snapshot.next:
            # --- CORRECTION MODE ---
            # If snapshot.next is not empty, the graph is PAUSED at the 'coach' interrupt.
            # We are submitting a REWRITE, not a new argument.
            
            print("   (Submitting correction to Referee...)")
            
            # A. Update the state with the fixed text
            app.update_state(config, {"latest_human_input": user_input})
            
            # B. Resume the graph (None tells it to continue execution)
            # The graph flows: Coach -> Referee (Re-Grade) -> Router
            events = app.stream(None, config)
            
        else:
            # --- NORMAL MODE ---
            # The graph is IDLE (at END). We are starting a fresh turn.
            
            # We pass the input to start the run from the top (Referee)
            events = app.stream(
                {"topic": topic, "latest_human_input": user_input}, 
                config
            )

        # 4. Handle the Output
        # We run the stream to let the graph execute
        for event in events:
            # Optional: You could print "Thinking..." here if you wanted
            pass
            
        # 5. Post-Execution Check
        # Did we finish successfully (Opponent spoke)? 
        # Or did we get paused again (Coach rejected)?
        snapshot = app.get_state(config)
        
        if snapshot.next:
            # PAUSED: The Referee rejected us again (or for the first time)
            score = snapshot.values.get('score')
            feedback = snapshot.values.get('feedback')
            
            print(f"\n✋ COACH INTERRUPT (Score: {score}/10)")
            print(f"💡 Feedback: {feedback}")
            print("👉 Try rewriting your argument based on this feedback.\n")
            # The loop restarts, and your next input will go to the 'Correction Mode' block
            
        else:
            # SUCCESS: The Opponent responded
            if 'messages' in snapshot.values and len(snapshot.values['messages']) > 0:
                last_msg = snapshot.values['messages'][-1]
                print(f"\n🤖 OPPONENT: {last_msg.content}\n")
            else:
                # Edge case handling
                print("Error: No response generated.")

# Run it
if __name__ == "__main__":
    chat_with_dojo()